In [ ]:
!pip install d2l
!pip install matplotlib-inline

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l
import torchvision as tv
import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [ ]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [ ]:
b1 = nn.Sequential(nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
                   nn.BatchNorm2d(16), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=1, padding=1))

In [ ]:
b2 = nn.Sequential(*resnet_block(16, 16, 3, first_block=True))
b3 = nn.Sequential(*resnet_block(16, 32, 3))
b4 = nn.Sequential(*resnet_block(32, 64, 3))

In [ ]:
net = nn.Sequential(b1, b2, b3, b4,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Dropout(0.1), nn.Linear(64, 10))

if device == 'cuda':
	net = torch.nn.DataParallel(net)

In [ ]:
X = torch.rand(size=(1, 3, 32, 32))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 16, 32, 32])
Sequential output shape:	 torch.Size([1, 16, 32, 32])
Sequential output shape:	 torch.Size([1, 32, 16, 16])
Sequential output shape:	 torch.Size([1, 64, 8, 8])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 64, 1, 1])
Flatten output shape:	 torch.Size([1, 64])
Dropout output shape:	 torch.Size([1, 64])
Linear output shape:	 torch.Size([1, 10])


In [ ]:
def evaluate_accuracy_gpu(net, data_iter, device=None): 
    if isinstance(net, nn.Module):
        net.eval()  
        if not device:
            device = next(iter(net.parameters())).device
    metric = d2l.Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [ ]:
def train_net(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            torch.nn.init.kaiming_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(params=net.parameters(),
                          lr=lr,
                          momentum=0.9,
                          weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(
        optimizer, milestones=[50, 90, 130], gamma=0.1)
    loss = nn.CrossEntropyLoss()
    num_batches = len(train_iter)
    for epoch in range(num_epochs):
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
        scheduler.step()
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        print(f'------- ------- Epoch{epoch+1}/{num_epochs} ------- -------')
        print(f'loss {train_l:.5f}, train acc {train_acc:.5f}, '
              f'test acc {test_acc:.5f}')

In [ ]:
shape_aug = tv.transforms.RandomResizedCrop(
    (28, 28), scale=(0.1, 1), ratio=(0.5, 2))

In [ ]:
train_augs = tv.transforms.Compose([
    tv.transforms.RandomCrop(32, padding=4),
    tv.transforms.RandomHorizontalFlip(),
    tv.transforms.ToTensor(), shape_aug,
    tv.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_augs = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
def load_cifar10(is_train, augs, batch_size):
    dataset = tv.datasets.CIFAR10(root="../data", train=is_train,
                                           transform=augs, download=True)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                    shuffle=is_train, num_workers=4)
    return dataloader

In [ ]:
lr, num_epochs, batch_size = 0.01, 150, 128

In [ ]:
train_iter = load_cifar10(True, train_augs, batch_size)
test_iter = load_cifar10(False, test_augs, batch_size)

In [ ]:
train_net(net, train_iter, test_iter, num_epochs, lr, device)

training on cuda
------- ------- Epoch1/150 ------- -------
loss 1.99659, train acc 0.26668, test acc 0.39980
------- ------- Epoch2/150 ------- -------
loss 1.76780, train acc 0.34798, test acc 0.38470
------- ------- Epoch3/150 ------- -------
loss 1.66481, train acc 0.39194, test acc 0.39080
------- ------- Epoch4/150 ------- -------
loss 1.59918, train acc 0.41770, test acc 0.46860
------- ------- Epoch5/150 ------- -------
loss 1.54562, train acc 0.44266, test acc 0.47070
------- ------- Epoch6/150 ------- -------
loss 1.49760, train acc 0.46194, test acc 0.49690
------- ------- Epoch7/150 ------- -------
loss 1.45974, train acc 0.47534, test acc 0.55370
------- ------- Epoch8/150 ------- -------
loss 1.41487, train acc 0.49520, test acc 0.56930
------- ------- Epoch9/150 ------- -------
loss 1.38477, train acc 0.50514, test acc 0.54410
------- ------- Epoch10/150 ------- -------
loss 1.36752, train acc 0.51356, test acc 0.58280
------- ------- Epoch11/150 ------- -------
loss 1.3